In [3]:
from Layers import FullyConnected, Helpers, Dropout, BatchNormalization, Flatten, RNN, Sigmoid, TanH, Initializers

In [2]:
batch_size = 9
input_size = 13
output_size = 5
hidden_size = 7
input_tensor = np.random.rand(input_size, batch_size).T

categories = 4
label_tensor = np.zeros([categories, batch_size]).T
for i in range(batch_size):
    label_tensor[i, np.random.randint(0, categories)] = 1

In [10]:
import numpy as np
from Layers.TanH import TanH
from Layers.Sigmoid import Sigmoid
from Layers.FullyConnected import FullyConnected
class RNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.fcy= FullyConnected(self.hidden_size, self.output_size)
        self.why = self.fcy.weights # np.random.uniform(size=(self.output_size, self.hidden_size))
        self.gradient_why = self.fcy._gradient_weights

        self.fcw = FullyConnected ( self.hidden_size + self.input_size, self.hidden_size)
        self.weights = self.fcw.weights #np.random.uniform(size=(self.hidden_size + self.input_size + 1, self.hidden_size))
        self._gradient_weights= self.fcw._gradient_weights

        self.hidden_state = np.zeros((1,hidden_size))
        self._memorize = False
        self.trainable = True
        self._optimizer = None

        self.hiddenstates = []
        self.outputs = []
        self.inputs = []


    @property
    def memorize(self):
        return self._memorize

    @memorize.setter
    def memorize(self, value):
        if isinstance(value, bool):
            self._memorize = value
        else:
            raise ValueError('memorize must be a boolean value.')

    @property
    def gradient_weights(self):
        # return the weights used in the hidden state computation
        return self.weights

    @gradient_weights.setter
    def gradient_weights(self, value):
        # set new value to the weights used in the hidden state computation
        self.weights = value

    def reset_state(self):
        # Reset stored inputs, outputs and hidden states to empty lists
        self.hiddenstates = []
        self.outputs = []
        self.inputs = []
        # Reset the inputs of fcw and fcy to None
        self.fcw.input = None
        self.fcy.input = None
        # Reset hidden_state to zeros
        self.hidden_state = np.zeros((1, self.hidden_size))

    def forward(self, input_tensor):
        if(not self._memorize):
            self.reset_state()
        time_steps = input_tensor.shape[0]  # Consider the “batch” dimension as the “time”

        output_tensor = np.zeros((time_steps, self.output_size))

        for t in range(time_steps):
            self.hiddenstates.append(self.hidden_state)
            self.inputs.append(input_tensor[t])

            x = input_tensor[t].reshape(1,-1)
            xs = np.concatenate((self.hidden_state,x), axis=1)
            self.fcw.input= xs
            self.hidden_state = TanH.forward(self,self.fcw.forward(xs))
            self.weights = self.fcw.weights

            output_tensor[t] = Sigmoid.forward(self,self.fcy.forward(self.hidden_state))
            self.why = self.fcy.weights

            self.outputs.append(output_tensor[t])
        return output_tensor

    def backward(self, error_tensor):
        self.time_steps = error_tensor.shape[0]
        self.out_error = np.zeros((self.time_steps, self.input_size))

        self.gradient_weights_y = np.zeros((self.hidden_size + 1, self.output_size))
        self.gradient_weights_h = np.zeros((self.hidden_size+self.input_size+1, self.hidden_size))

        count = 0

        grad_tanh = 1-self.h_t[1::] ** 2
        hidden_error = np.zeros((1, self.hidden_size))
        
        for b in reversed(range(self.time_steps)):
            yh_error = self.FC_y.backward(error_tensor[b][np.newaxis, :])
            self.FC_y.input_tensor = np.hstack((self.h_t[b+1], 1))[np.newaxis, :]

            grad_yh = hidden_error + yh_error
            grad_hidden = grad_tanh[b]*grad_yh
            xh_error = self.FC_h.backward(grad_hidden)
            hidden_error = xh_error[:, 0:self.hidden_size]
            x_error = xh_error[:, self.hidden_size:(self.hidden_size + self.input_size + 1)]
            self.out_error[b] = x_error

            con = np.hstack((self.h_t[b], self.input_tensor[b],1))
            self.FC_h.input_tensor = con[np.newaxis, :]
            # print(count, "", self.bptt)
            if count <= self.bptt:
                self.weights_y = self.FC_y.weights
                self.weights_h = self.FC_h.weights
                self.gradient_weights_y = self.FC_y.gradient_weights
                self.gradient_weights_h = self.FC_h.gradient_weights
            count += 1

        if self.optimizer is not None:
            self.weights_y = self.optimizer.calculate_update(self.weights_y, self.gradient_weights_y)
            self.weights_h = self.optimizer.calculate_update(self.weights_h, self.gradient_weights_h)
            self.FC_y.weights = self.weights_y
            self.FC_h.weights = self.weights_h
        return self.out_error


    def calculate_regularization_loss(self):
        # if no regularizer is set, just return 0
        if self.optimizer is None or self.optimizer.regularizer is None:
            return 0
        # otherwise, calculate the regularization loss based on the weights
        return self.optimizer.regularizer.norm(self.weights)

    def initialize(self, weights_initializer, bias_initializer):
        self.fcw.weights = weights_initializer.initialize(self.weights.shape, self.input_size, self.hidden_size)
        self.fcy.weights = weights_initializer.initialize(self.why.shape, self.hidden_size, self.output_size)
        self.why = self.fcy.weights
        self.weights = self.fcw.weights


    @property
    def optimizer(self):
        return self._optimizer

    @optimizer.setter
    def optimizer(self, optimizer):
        self._optimizer = optimizer
        self._optimizer.layer = self

In [11]:
layer = RNN(input_size, hidden_size, output_size)
output_tensor = layer.forward(input_tensor)
error_tensor = layer.backward(output_tensor)

AttributeError: 'RNN' object has no attribute 'h_t'